# hVAE

In [3]:
from src.hvae import hVAE
from hparams import get_hparams

PATH = "path"
dataset = get_hparams().data_params.dataset
test_loader = dataset.get_test_loader()

hvae = hVAE.load(PATH)
hvae.test_model(test_loader)
hvae.generate()

TypeError: load() missing 1 required positional argument: 'path'